In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd

from utils import (
    read_single_problem_from_path_as_adjacency,
    read_single_problem_from_path_as_sparse,
    read_single_problem_from_path_as_sparse_from_adjacency,
    order_nodes_in_descending_order,
    check_validity_for_adjacency,
    transform_node_clique_to_zero_one,
    transform_zero_one_clique_to_nodes
)
from constants import BASE_INSTANCES_FILES
from heuristics import (
    descending_degree_glutonous_heuristic,
    dynamic_descending_degree_glutonous_heuristic,
    descending_degree_random_heuristic,
    multiple_descending_degree_random_heuristic
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
print(graph)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))


[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]
[0 1 0 0 1 0 1 1 0 1]
Is the clique valid ?  True


In [5]:
# Check that clique representation utils work
number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print(transform_zero_one_clique_to_nodes(clique))
print(transform_node_clique_to_zero_one(number_of_nodes, transform_zero_one_clique_to_nodes(clique)))

[0 1 0 0 1 0 1 1 0 1]
[1, 4, 6, 7, 9]
[0 1 0 0 1 0 1 1 0 1]


In [4]:
# Build clique size and times of execution for our base glutonous heuristics
files = []
instance_reading_times = []
clique_creating_times = []
total_times = []
clique_sizes = []
methods = []

for file in BASE_INSTANCES_FILES:
    for method, verbose_method in [
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ]:
        # Method and file adding
        methods.append(verbose_method)
        files.append(file)

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=BASE_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = method(graph=graph, degrees=degrees)
        clique_sizes.append(np.sum(clique))
        end_of_clique_time = time()
        clique_creating_times.append(end_of_clique_time - end_of_read_time)

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, clique_creating_times, total_times, clique_sizes, methods]:
        display_list.append("/////")


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# Display for most basic heuristics

display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "instance time": instance_reading_times,
        "clique time": clique_creating_times,
        "total time": total_times,
    }
)
print(display_dataframe)


              file     method clique size instance time clique time total time
0   brock200_2.col  desc_base           7      0.054014    0.004027   0.058041
1   brock200_2.col   desc_dyn           7      0.051069    0.003986   0.055055
2   brock200_2.col   desc_ran           7      0.054015    0.004947   0.058962
3   brock200_2.col   mult_ran           8         0.054    0.408984   0.462984
4            /////      /////       /////         /////       /////      /////
5    dsjc125.1.col  desc_base           4      0.005032    0.002007   0.007039
6    dsjc125.1.col   desc_dyn           4         0.003    0.002001   0.005001
7    dsjc125.1.col   desc_ran           3      0.004009    0.001991      0.006
8    dsjc125.1.col   mult_ran           3      0.004012    0.185763   0.189775
9            /////      /////       /////         /////       /////      /////
10   random-10.col  desc_base           5           0.0         0.0        0.0
11   random-10.col   desc_dyn           5      0.001

In [ ]:
# Test different size of choices for the nodes for random heuristic
files = []
instance_reading_times = []
clique_creating_times = []
total_times = []
clique_sizes = []
methods = []

for file in ["brock200_2.col"]:
    for k in range(1, 11):
        # Method and file adding
        methods.append(f"multi_ran_{k}")
        files.append(file)

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=BASE_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = multiple_descending_degree_random_heuristic(graph=graph, degrees=degrees, size_of_choice=k, number_of_iterations=20)
        clique_sizes.append(len(clique))
        end_of_clique_time = time()
        clique_creating_times.append(end_of_clique_time - end_of_read_time)

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, clique_creating_times, total_times, clique_sizes, methods]:
        display_list.append("/////")


In [ ]:
# Display for different size of choices for random heuristic

display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "instance time": instance_reading_times,
        "clique time": clique_creating_times,
        "total time": total_times,
    }
)
print(display_dataframe)

              file        method clique size instance time clique time  \
0   brock200_2.col   multi_ran_1           7      0.063993    0.090002   
1   brock200_2.col   multi_ran_2           7      0.050018    0.083028   
2   brock200_2.col   multi_ran_3           7      0.052954    0.090038   
3   brock200_2.col   multi_ran_4           7      0.058967    0.119515   
4   brock200_2.col   multi_ran_5           8      0.111997    0.113998   
5   brock200_2.col   multi_ran_6           7      0.051999    0.083002   
6   brock200_2.col   multi_ran_7           8         0.059    0.090137   
7   brock200_2.col   multi_ran_8           8      0.052107    0.082648   
8   brock200_2.col   multi_ran_9           8      0.052999       0.087   
9   brock200_2.col  multi_ran_10           8      0.055054    0.081987   
10           /////         /////       /////         /////       /////   

   total time  
0    0.153996  
1    0.133045  
2    0.142992  
3    0.178482  
4    0.225995  
5    0.136001  